In [61]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習を勉強中です。'

for token in j_t.tokenize(text):
    print(token)
    
# wakati　＝Trueで、単語のみを抽出する
def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

print(tokenizer_janome(text))

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
勉強	名詞,サ変接続,*,*,*,*,勉強,ベンキョウ,ベンキョー
中	名詞,接尾,副詞可能,*,*,*,中,チュウ,チュー
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
['機械', '学習', 'を', '勉強', '中', 'です', '。']


# MeCab

In [2]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')
text = '機械学習を勉強中です。'

print(m_t.parse(text))

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
を	ヲ	を	助詞-格助詞-一般		
勉強	ベンキョウ	勉強	名詞-サ変接続		
中	チュウ	中	名詞-接尾-副詞可能		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [8]:
m_t = MeCab.Tagger('-Ochasen -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
text = '機械学習を勉強中です。'

print(m_t.parse(text))

機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
勉強	ベンキョウ	勉強	名詞-サ変接続		
中	チュウ	中	名詞-接尾-副詞可能		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [12]:
m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret

text = '機械学習を勉強中です。'
print(tokenizer_mecab(text))

['機械学習', 'を', '勉強', '中', 'です', '。']


# 1 . 前処理と単語分割の関数を実装

In [62]:
# 単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer
import re

j_t = Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

def preprocessing_text(text):
    # 半角・全角の統一
    # 今回は無視

    # 英語の小文字化
    # 今回はここでは無視
    # output = output.lower()

    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    
    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 0-9]', '0', text)
    
    # 記号と数字の除去
    # 今回は無視。半角記号,数字,英字
    # 今回は無視。全角記号

    # 特定文字を正規表現で置換する
    # 今回は無視
    
    return text

# 前処理とJanomeの単語分割を合わせた関数を定義する
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_janome(text)
    
    return ret

# 動作確認
text = "昨日は とても暑く、気温が36度もあった。"
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


# 2. 文章データの読み込み

In [63]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


In [64]:
# data.TabularDataset 詳細
# https://torchtext.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits

# フォルダ「data」から各tsvファイルを読み込み、Datasetにします
# 1行がTEXTとLABELで区切られていることをfieldsで指示します
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


# 動作確認
print('訓練データの数', len(train_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))


訓練データの数 4
1つ目の訓練データ {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
2つ目の訓練データ {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


# 単語の数値化

In [65]:
# ボキャブラリーを作成します
# 訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー（単語集）を構築
TEXT.build_vocab(train_ds, min_freq=1)
# 訓練データ内の単語と頻度を出力(頻度min_freqより大きいものが出力されます)
TEXT.vocab.freqs  # 出力させる

Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         '、': 3,
         'その': 1,
         'ネガティブ': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'ポジティブ': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [66]:
# ボキャブラリーの単語をidに変換した結果を出力。
# 頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi  # 出力。string to identifiers 文字列をidへ


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fc249aa59b0>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
     

# DataLoaderの作成

In [76]:
# 単語分割にはMecab＋NEologdを使用
import MeCab
import torchtext

m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret


# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


# 2. 単語のベクトル化
## 2.1 word2vec

In [78]:
from gensim.models import KeyedVectors

# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format('data/entity_vector/entity_vector.model.bin', binary=True)

model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')

/home/nagae/.conda/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [70]:
from torchtext.vocab import Vectors
# torchtextで単語ベクトルとして読み込みます

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))

100%|█████████▉| 1015466/1015474 [01:08<00:00, 15013.11it/s]

1単語を表現する次元数： 200
単語数： 1015474


100%|█████████▉| 1015466/1015474 [01:20<00:00, 15013.11it/s]

In [79]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors

torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

In [88]:
# ボキャブラリーの単語の順番を確認します
print(TEXT.vocab.stoi)


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fc21a16dbe0>>, {'<unk>': 0, '<pad>': 1, 'と': 2, '。': 3, 'な': 4, 'の': 5, '文章': 6, '、': 7, 'が': 8, 'し': 9, 'を': 10, 'いる': 11, 'か': 12, 'て': 13, 'ます': 14, '分類': 15, '本章': 16, '評価': 17, '0': 18, 'い': 19, 'から': 20, 'する': 21, 'その': 22, 'た': 23, 'で': 24, 'です': 25, 'に': 26, 'に対して': 27, 'は': 28, 'まし': 29, 'クラス': 30, 'ネガティブ': 31, 'ポジティブ': 32, 'モデル': 33, 'レビュー': 34, '値': 35, '取り組み': 36, '商品': 37, '女性': 38, '女王': 39, '好き': 40, '姫': 41, '構築': 42, '機械学習': 43, '王': 44, '王子': 45, '男性': 46, '短い': 47, '自然言語処理': 48})


In [81]:
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


## 2.2 fastText

In [83]:
from torchtext.vocab import Vectors
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))


1単語を表現する次元数： 300
単語数： 351122


In [87]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
print(TEXT.vocab.vectors)
# ボキャブラリーの単語の順番を確認します
print(TEXT.vocab.stoi)

torch.Size([49, 300])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0182, -0.0894, -0.1011,  ..., -0.0410, -0.0735,  0.0045],
        ...,
        [ 0.0161, -0.0172, -0.2945,  ..., -0.0038,  0.1588, -0.0076],
        [-0.2099,  0.1631, -0.0182,  ..., -0.4823,  0.0375,  0.0088],
        [-0.2762,  0.2986,  0.0511,  ..., -0.1052, -0.1779, -0.1957]])
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fc21a16dbe0>>, {'<unk>': 0, '<pad>': 1, 'と': 2, '。': 3, 'な': 4, 'の': 5, '文章': 6, '、': 7, 'が': 8, 'し': 9, 'を': 10, 'いる': 11, 'か': 12, 'て': 13, 'ます': 14, '分類': 15, '本章': 16, '評価': 17, '0': 18, 'い': 19, 'から': 20, 'する': 21, 'その': 22, 'た': 23, 'で': 24, 'です': 25, 'に': 26, 'に対して': 27, 'は': 28, 'まし': 29, 'クラス': 30, 'ネガティブ': 31, 'ポジティブ': 32, 'モデル': 33, 'レビュー': 34, '値': 35, '取り組み': 36, '商品': 37, '女性': 38, '女王': 39, '好き': 40, '姫': 41, '構築': 42, '機械学習': 43, '王':

In [89]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)


# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成

In [97]:
# tsv形式のファイルにします
import glob
import os
import io
import string

# 訓練データのtsvファイルを作成します
f = open('./data/IMDb_train.tsv', 'w')

# positiveデータ
path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        
        # タブがあれば消しておきます
        text = text.replace('\t', ' ')
        
        text = text + '\t' + '1' + '\t' + '\n'
        f.write(text)
        
# negative データ
path = './data/aclImdb/test/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)
        
f.close()


# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [93]:
# 2. 前処理と単語分割の関数を定義

In [95]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)

def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)
    
    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p=='.') or (p==','):
            continue
            
        else:
            text = text.replace(p, ' ')
    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace('.', ' . ')
    text = text.replace(',', ' , ')
    
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）

def tokenizer_punctuation(text):
    return text.strip().split()

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats!.'))

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


# DataLoaderの作成

In [99]:
# 文章とラベルの両方に用意します
max_length = 256

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))



訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['i', 'must', 'say', ',', 'i', 'was', 'surprised', 'with', 'the', 'quality', 'of', 'the', 'movie', '.', 'it', 'was', 'far', 'better', 'than', 'i', 'expected', '.', 'scenario', 'and', 'acting', 'is', 'quite', 'good', '.', 'the', 'director', 'made', 'a', 'good', 'job', 'as', 'well', '.', 'although', 'some', 'scenes', 'look', 'a', 'bit', 'clumsy', ',', 'it', 'is', 'a', 'decent', 'movie', 'overall', '.', 'the', 'idea', 'was', 'definitely', 'brilliant', 'and', 'the', 'truth', 'did', 'not', 'reveal', 'itself', 'till', 'the', 'very', 'end', '.', 'the', 'mental', 'hospital', 'atmosphere', 'was', 'given', 'quite', 'good', '.', 'the', 'plot', 'was', 'clear', ',', 'consistent', 'and', 'well', 'thought', '.', 'some', 'people', 'may', 'find', 'it', 'a', 'bit', 'boring', 'though', 'since', 'the', 'story', 'line', 'is', 'very', 'focused', 'and', 'they', 'take', 'their', 'time', 'for', 'character', 'and', 'story', 'development', '.', 'moral', 'of', 'the', 's

In [100]:
import random

# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['the', 'movie', 'opens', 'with', 'charlie', 'jeff', 'daniels', ',', 'a', 'business', 'man', 'just', 'finishing', 'his', 'lunch', 'in', 'a', 'neighborhood', 'deli', '.', 'it', 'appears', 'he', 'doesn', 't', 'have', 'enough', 'cash', 'to', 'cover', 'the', 'check', '.', 'instead', 'of', 'reaching', 'for', 'his', 'plastic', ',', 'he', 'furtively', 'glances', 'around', 'to', 'see', 'if', 'the', 'coast', 'is', 'clear', 'and', 'ducks', 'out', 'of', 'the', 'place', 'without', 'paying', '.', 'unbeknownst', 'to', 'charlie', ',', 'lulu', 'melanie', 'griffith', 'had', 'been', 'observing', 'him', 'from', 'the', 'other', 'side', 'of', 'the', 'deli', '.', 'lulu', 'is', 'decked', 'out', 'in', 'what', 'passed', 'for', 'cool', 'back', 'in', 'the', '1980', 's', 'with', 'a', 'brunette', 'page', 'boy', 'cut', '.', 'she', 'follows', 'him', 'out', 'to', 'let', 'him', 'know', 'that', 'she', 'saw', 'what', 'he', 'did', '.', 'he', 'tries', 'to', 'deny', 'it', 'but'

# ボキャブラリーを作成

In [101]:
from torchtext.vocab import Vectors
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))



  0%|          | 0/999994 [00:00<?, ?it/s]Skipping token b'999994' with 1-dimensional vector [b'300']; likely a header


  0%|          | 1114/999994 [00:00<01:29, 11139.61it/s]

  0%|          | 2239/999994 [00:00<01:29, 11167.91it/s]

  0%|          | 3344/999994 [00:00<01:29, 11130.18it/s]

  0%|          | 4453/999994 [00:00<01:29, 11117.04it/s]

  1%|          | 5561/999994 [00:00<01:29, 11104.36it/s]

  1%|          | 6667/999994 [00:00<01:29, 11088.13it/s]

  1%|          | 7775/999994 [00:00<01:29, 11084.37it/s]

  1%|          | 8885/999994 [00:00<01:29, 11086.27it/s]

  1%|          | 9991/999994 [00:00<01:29, 11077.71it/s]

  1%|          | 11098/999994 [00:01<01:29, 11072.93it/s]

  1%|          | 12207/999994 [00:01<01:29, 11076.88it/s]

  1%|▏         | 13316/999994 [00:01<01:29, 11080.43it/s]

  1%|▏         | 14424/999994 [00:01<01:28, 11079.44it/s]

  2%|▏         | 15558/999994 [00:01<01:28, 11153.69it/s]

  2%|▏         | 16690/999994 [00:01<01:27, 11201.45it/s]

 

 14%|█▍        | 139010/999994 [00:12<01:17, 11095.54it/s]

 14%|█▍        | 140120/999994 [00:12<01:17, 11071.12it/s]

 14%|█▍        | 141228/999994 [00:12<01:17, 11052.42it/s]

 14%|█▍        | 142334/999994 [00:12<01:17, 11033.52it/s]

 14%|█▍        | 143438/999994 [00:12<01:17, 11021.79it/s]

 14%|█▍        | 144563/999994 [00:12<01:17, 11087.56it/s]

 15%|█▍        | 145693/999994 [00:13<01:16, 11147.92it/s]

 15%|█▍        | 146820/999994 [00:13<01:16, 11183.11it/s]

 15%|█▍        | 147946/999994 [00:13<01:16, 11204.49it/s]

 15%|█▍        | 149073/999994 [00:13<01:15, 11224.04it/s]

 15%|█▌        | 150201/999994 [00:13<01:15, 11239.76it/s]

 15%|█▌        | 151330/999994 [00:13<01:15, 11254.65it/s]

 15%|█▌        | 152456/999994 [00:13<01:15, 11255.84it/s]

 15%|█▌        | 153582/999994 [00:13<01:15, 11256.48it/s]

 15%|█▌        | 154708/999994 [00:13<01:15, 11179.72it/s]

 16%|█▌        | 155834/999994 [00:13<01:15, 11201.27it/s]

 16%|█▌        | 156959/999994 [00:14<01

 28%|██▊       | 278150/999994 [00:24<01:04, 11206.02it/s]

 28%|██▊       | 279279/999994 [00:25<01:04, 11229.04it/s]

 28%|██▊       | 280403/999994 [00:25<01:04, 11184.22it/s]

 28%|██▊       | 281535/999994 [00:25<01:04, 11223.37it/s]

 28%|██▊       | 282669/999994 [00:25<01:03, 11257.40it/s]

 28%|██▊       | 283805/999994 [00:25<01:03, 11285.59it/s]

 28%|██▊       | 284940/999994 [00:25<01:03, 11303.74it/s]

 29%|██▊       | 286074/999994 [00:25<01:03, 11313.76it/s]

 29%|██▊       | 287207/999994 [00:25<01:02, 11317.11it/s]

 29%|██▉       | 288342/999994 [00:25<01:02, 11324.81it/s]

 29%|██▉       | 289475/999994 [00:25<01:03, 11263.22it/s]

 29%|██▉       | 290602/999994 [00:26<01:03, 11210.10it/s]

 29%|██▉       | 291738/999994 [00:26<01:02, 11252.97it/s]

 29%|██▉       | 292864/999994 [00:26<01:03, 11182.66it/s]

 29%|██▉       | 293983/999994 [00:26<01:03, 11160.70it/s]

 30%|██▉       | 295100/999994 [00:26<01:03, 11136.50it/s]

 30%|██▉       | 296214/999994 [00:26<01

 42%|████▏     | 419006/999994 [00:37<00:51, 11380.91it/s]

 42%|████▏     | 420146/999994 [00:37<00:50, 11385.98it/s]

 42%|████▏     | 421290/999994 [00:37<00:50, 11400.22it/s]

 42%|████▏     | 422431/999994 [00:37<00:51, 11321.18it/s]

 42%|████▏     | 423568/999994 [00:37<00:50, 11333.78it/s]

 42%|████▏     | 424710/999994 [00:37<00:50, 11357.09it/s]

 43%|████▎     | 425852/999994 [00:37<00:50, 11375.09it/s]

 43%|████▎     | 426990/999994 [00:38<00:50, 11316.18it/s]

 43%|████▎     | 428122/999994 [00:38<00:50, 11272.05it/s]

 43%|████▎     | 429262/999994 [00:38<00:50, 11308.41it/s]

 43%|████▎     | 430402/999994 [00:38<00:50, 11332.95it/s]

 43%|████▎     | 431541/999994 [00:38<00:50, 11348.80it/s]

 43%|████▎     | 432684/999994 [00:38<00:49, 11372.02it/s]

 43%|████▎     | 433822/999994 [00:38<00:51, 11091.65it/s]

 43%|████▎     | 434934/999994 [00:38<00:50, 11099.51it/s]

 44%|████▎     | 436071/999994 [00:38<00:50, 11178.50it/s]

 44%|████▎     | 437212/999994 [00:38<00

 56%|█████▌    | 559356/999994 [00:49<00:40, 10923.09it/s]

 56%|█████▌    | 560452/999994 [00:49<00:40, 10727.42it/s]

 56%|█████▌    | 561593/999994 [00:50<00:40, 10922.19it/s]

 56%|█████▋    | 562738/999994 [00:50<00:39, 11073.36it/s]

 56%|█████▋    | 563848/999994 [00:50<00:39, 11064.42it/s]

 56%|█████▋    | 564993/999994 [00:50<00:38, 11175.93it/s]

 57%|█████▋    | 566137/999994 [00:50<00:38, 11251.41it/s]

 57%|█████▋    | 567264/999994 [00:50<00:39, 10934.51it/s]

 57%|█████▋    | 568361/999994 [00:50<00:40, 10761.25it/s]

 57%|█████▋    | 569440/999994 [00:50<00:40, 10627.70it/s]

 57%|█████▋    | 570505/999994 [00:50<00:40, 10526.88it/s]

 57%|█████▋    | 571560/999994 [00:51<00:41, 10440.20it/s]

 57%|█████▋    | 572703/999994 [00:51<00:39, 10716.76it/s]

 57%|█████▋    | 573778/999994 [00:51<00:40, 10575.63it/s]

 57%|█████▋    | 574838/999994 [00:51<00:40, 10474.44it/s]

 58%|█████▊    | 575888/999994 [00:51<00:40, 10409.62it/s]

 58%|█████▊    | 576931/999994 [00:51<00

 70%|██████▉   | 696538/999994 [01:02<00:28, 10665.51it/s]

 70%|██████▉   | 697608/999994 [01:02<00:28, 10559.84it/s]

 70%|██████▉   | 698667/999994 [01:02<00:28, 10476.99it/s]

 70%|██████▉   | 699717/999994 [01:02<00:28, 10430.28it/s]

 70%|███████   | 700824/999994 [01:03<00:28, 10612.72it/s]

 70%|███████   | 701967/999994 [01:03<00:27, 10843.55it/s]

 70%|███████   | 703054/999994 [01:03<00:27, 10682.54it/s]

 70%|███████   | 704125/999994 [01:03<00:28, 10507.70it/s]

 71%|███████   | 705262/999994 [01:03<00:27, 10750.72it/s]

 71%|███████   | 706340/999994 [01:03<00:27, 10624.69it/s]

 71%|███████   | 707462/999994 [01:03<00:27, 10794.07it/s]

 71%|███████   | 708544/999994 [01:03<00:27, 10565.24it/s]

 71%|███████   | 709604/999994 [01:03<00:27, 10468.24it/s]

 71%|███████   | 710653/999994 [01:03<00:27, 10439.57it/s]

 71%|███████   | 711699/999994 [01:04<00:27, 10425.68it/s]

 71%|███████▏  | 712743/999994 [01:04<00:27, 10420.66it/s]

 71%|███████▏  | 713786/999994 [01:04<00

 83%|████████▎ | 832085/999994 [01:15<00:15, 10735.48it/s]

 83%|████████▎ | 833162/999994 [01:15<00:15, 10624.19it/s]

 83%|████████▎ | 834227/999994 [01:15<00:15, 10472.66it/s]

 84%|████████▎ | 835277/999994 [01:15<00:15, 10367.59it/s]

 84%|████████▎ | 836316/999994 [01:15<00:15, 10361.36it/s]

 84%|████████▎ | 837354/999994 [01:15<00:15, 10348.81it/s]

 84%|████████▍ | 838390/999994 [01:15<00:15, 10332.22it/s]

 84%|████████▍ | 839424/999994 [01:16<00:15, 10332.35it/s]

 84%|████████▍ | 840461/999994 [01:16<00:15, 10342.53it/s]

 84%|████████▍ | 841498/999994 [01:16<00:15, 10350.05it/s]

 84%|████████▍ | 842634/999994 [01:16<00:14, 10633.47it/s]

 84%|████████▍ | 843700/999994 [01:16<00:14, 10542.47it/s]

 84%|████████▍ | 844842/999994 [01:16<00:14, 10791.23it/s]

 85%|████████▍ | 845989/999994 [01:16<00:14, 10985.62it/s]

 85%|████████▍ | 847132/999994 [01:16<00:13, 11114.60it/s]

 85%|████████▍ | 848275/999994 [01:16<00:13, 11206.34it/s]

 85%|████████▍ | 849398/999994 [01:16<00

 97%|█████████▋| 967235/999994 [01:28<00:03, 10425.59it/s]

 97%|█████████▋| 968280/999994 [01:28<00:03, 10402.98it/s]

 97%|█████████▋| 969322/999994 [01:28<00:02, 10387.44it/s]

 97%|█████████▋| 970442/999994 [01:28<00:02, 10617.44it/s]

 97%|█████████▋| 971506/999994 [01:28<00:02, 10454.35it/s]

 97%|█████████▋| 972554/999994 [01:28<00:02, 10365.25it/s]

 97%|█████████▋| 973672/999994 [01:28<00:02, 10596.67it/s]

 97%|█████████▋| 974803/999994 [01:28<00:02, 10800.83it/s]

 98%|█████████▊| 975943/999994 [01:28<00:02, 10971.56it/s]

 98%|█████████▊| 977043/999994 [01:28<00:02, 10760.23it/s]

 98%|█████████▊| 978122/999994 [01:29<00:02, 10597.92it/s]

 98%|█████████▊| 979185/999994 [01:29<00:01, 10476.93it/s]

 98%|█████████▊| 980308/999994 [01:29<00:01, 10689.75it/s]

 98%|█████████▊| 981426/999994 [01:29<00:01, 10830.91it/s]

 98%|█████████▊| 982512/999994 [01:29<00:01, 10569.30it/s]

 98%|█████████▊| 983572/999994 [01:29<00:01, 10393.69it/s]

 98%|█████████▊| 984693/999994 [01:29<00

1単語を表現する次元数： 300
単語数： 999994




100%|█████████▉| 999935/999994 [01:47<00:00, 10465.36it/s]

In [102]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
print(TEXT.vocab.vectors)

# ボキャブラリーの単語の順番を確認します
print(TEXT.vocab.stoi)

torch.Size([17800, 300])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1276,  0.0781,  0.1026,  ...,  0.1987, -0.2986, -0.0822],
        [ 0.0147, -0.0415,  0.0006,  ...,  0.0387, -0.0181, -0.0128]])
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fc1fe31d470>>, {'<unk>': 0, '<pad>': 1, '<cls>': 2, '<eos>': 3, 'the': 4, '.': 5, ',': 6, 'a': 7, 'and': 8, 'of': 9, 'to': 10, 'is': 11, 'it': 12, 'in': 13, 'i': 14, 'this': 15, 'that': 16, 's': 17, 'was': 18, 'as': 19, 'movie': 20, 'with': 21, 'for': 22, 'but': 23, 'film': 24, 'you': 25, 'on': 26, 't': 27, 'not': 28, 'he': 29, 'are': 30, 'his': 31, 'have': 32, 'be': 33, 'one': 34, 'all': 35, 'at': 36, 'they': 37, 'by': 38, 'an': 39, 'who': 40, 'f

In [123]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)
print(batch.Text[0].shape)

(tensor([[   2,   74,   12,  ..., 1059,   87,    3],
        [   2,    4,   67,  ...,  117,  609,    3],
        [   2, 1207,  132,  ...,    1,    1,    1],
        ...,
        [   2,    4,    0,  ...,    1,    1,    1],
        [   2,    4, 1340,  ...,   15,   34,    3],
        [   2,   14,  839,  ...,    1,    1,    1]]), tensor([256, 256, 160, 253, 225, 182, 249, 256, 138, 256, 152, 256, 217, 256,
        256,  60, 220, 220, 152, 149, 256, 145, 256,  68]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])
torch.Size([24, 256])


# Transformerモデル（分類タスク用）の実装

In [125]:
import math
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchtext

# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [127]:
class Embedder(nn.Module):
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        
        # freeze=Trueによりバックプロパゲーションで更新されず変化しなくなります
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        
    def forward(self, x):
        x_vec = self.embeddings(x)
        
        return x_vec

In [129]:
# 前節のDataLoaderなどを取得
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=24)

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)

# 入出力
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに

print("入力のテンソルサイズ：", x.shape)
print("出力のテンソルサイズ：", x1.shape)

入力のテンソルサイズ： torch.Size([24, 256])
出力のテンソルサイズ： torch.Size([24, 256, 300])


In [138]:
class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''
    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()
        
        self.d_model = d_model
        
        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)
        
        # GPUが使える場合はGPUへ送る、ここでは省略。実際に学習時には使用する
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i)/d_model)))
                
        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)
        
        # 勾配を計算しないようにする
        self.requires_grad = False
        
    def forward(self, x):
        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

In [139]:
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)

x = batch.Text[0]

x1 = net1(x) # 単語をベクトルに
x2 = net2(x1)

print("元のテンソルサイズ：", x.shape)
print("入力のテンソルサイズ：", x1.shape)
print("出力のテンソルサイズ：", x2.shape)


元のテンソルサイズ： torch.Size([24, 256])
入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ： torch.Size([24, 256, 300])


In [149]:
class Attention(nn.Module):
    '''Transformerは本当はマルチヘッドAttentionですが、
    分かりやすさを優先しシングルAttentionで実装します'''

    def __init__(self, d_model=300):
        super().__init__()

        # SAGANでは1dConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)

        # Attentionの大きさ調整の変数
        self.d_k = d_model

    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)

        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        normlized_weights = F.softmax(weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)

        # 全結合層で特徴量を変換
        output = self.out(output)

        return output, normlized_weights


In [150]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        '''Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニットです'''
        super().__init__()
        
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(x)
        x = self.linear_2(x)
        return x

In [151]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        
        # LayerNormalization層
        # https://pytorch.org/docs/stable/nn.html?highlight=layernorm
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        
        # Attention層
        self.attention = Attention(d_model)
        
        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)
        
        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        # 正規化とAttention
        x_normalized = self.norm_1(x)
        output, normalized_weights = self.attention(x_normalized, x_normalized, x_normalized, mask)
        
        x2 = x + self.dropout_1(output)
        
        # 正規化と全結合層
        x_normalized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normalized2))
        
        return output, normalized_weights

In [152]:
# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)

# maskの作成
x = batch.Text[0]
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (x != input_pad) *1
print(input_mask[0])

# 入出力
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換

print("入力のテンソルサイズ：", x2.shape)
print("出力のテンソルサイズ：", x3.shape)
print("Attentionのサイズ：", normlized_weights.shape)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ： torch.Size([24, 256, 300])
Attentionのサイズ： torch.Size([24, 256, 256])


In [159]:
class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''
    def __init__(self, d_model=300, output_dim=2):
        super().__init__()
        
        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # output_dimはポジ・ネガの2つ
        
        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, std=0.2)
        
    def forward(self, x):
        x0 = x[:, 0, :] # 各ミニバッチの各文の先頭の単語の特徴量（300次元）を取り出す
        out = self.linear(x0)
        return out

In [160]:
# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)

# 入出力
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換
x4 = net4(x3)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力

print("入力のテンソルサイズ：", x3.shape)
print("出力のテンソルサイズ：", x4.shape)


入力のテンソルサイズ： torch.Size([24, 256, 300])
出力のテンソルサイズ： torch.Size([24, 2])


## 最終的なTransformerモデル

In [167]:
class TransformerClassification(nn.Module):
    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()
        
        # モデル構築
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)
        
    def forward(self, x, mask):
        x1 = self.net1(x)  # 単語をベクトルに
        x2 = self.net2(x1)  # Positon情報を足し算
        x3_1, normlized_weights_1 = self.net3_1(
            x2, mask)  # Self-Attentionで特徴量を変換
        x3_2, normlized_weights_2 = self.net3_2(
            x3_1, mask)  # Self-Attentionで特徴量を変換
        x4 = self.net4(x3_2)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
        return x4, normlized_weights_1, normlized_weights_2


In [168]:
# 動作確認

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# 入出力
x = batch.Text[0]
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x, input_mask)

print("出力のテンソルサイズ：", out.shape)
print("出力テンソルのsigmoid：", F.softmax(out, dim=1))


出力のテンソルサイズ： torch.Size([24, 2])
出力テンソルのsigmoid： tensor([[0.4575, 0.5425],
        [0.4910, 0.5090],
        [0.4635, 0.5365],
        [0.4634, 0.5366],
        [0.4660, 0.5340],
        [0.5362, 0.4638],
        [0.3954, 0.6046],
        [0.4963, 0.5037],
        [0.4852, 0.5148],
        [0.4271, 0.5729],
        [0.4304, 0.5696],
        [0.4802, 0.5198],
        [0.4440, 0.5560],
        [0.5211, 0.4789],
        [0.4522, 0.5478],
        [0.4259, 0.5741],
        [0.5244, 0.4756],
        [0.5412, 0.4588],
        [0.4362, 0.5638],
        [0.4031, 0.5969],
        [0.4790, 0.5210],
        [0.4637, 0.5363],
        [0.4764, 0.5236],
        [0.4198, 0.5802]], grad_fn=<SoftmaxBackward>)


# 7.7 Transformerの学習・推論、判定根拠の可視化を実装

In [170]:
# パッケージのimport
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DatasetとDataLoaderを作成

In [171]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=64)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


# ネットワークモデルの作成

In [ ]:
from utils.transformer import TransformerClassification

net = TransformerClassification(text_embedding_vectors=TEXT, )